In [ ]:
!pip install patool

In [2]:
import tensorflow as tf

import tensorflow.keras.utils as utils

import tensorflow.keras.applications as applications

import patoolib, os, warnings

warnings.filterwarnings("ignore")

unrar = lambda url : patoolib.extract_archive(url, outdir=".")

In [ ]:
url = "/content/inputer.rar"

unrar(url)

In [4]:
rooter = "/content/image"

classes = os.listdir(rooter)

classes

['pria', 'perempuan']

In [5]:
i = len(os.listdir(os.path.join(rooter, "pria")))

j = len(os.listdir(os.path.join(rooter, "perempuan")))

total = i + j

total

878

In [ ]:
subset = "both"; label = "binary"

image = (160, 160); split = 0.15; random = 18

orifile = utils.image_dataset_from_directory(
  rooter,
  seed = random,
  subset = subset,
  image_size = image,
  label_mode = label,
  batch_size = random,
  validation_split = split
)

train, testi = orifile

In [ ]:
input_shape = (160, 160, 3)

ontrue = !False

intnet = applications.EfficientNetB5(weights="imagenet", include_top=False, input_shape=input_shape)

for layer in intnet.layers[15:] : layer.trainable = ontrue

In [8]:
intnet.inputs[0]

<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>

In [9]:
intnet.outputs[0]

<KerasTensor: shape=(None, 5, 5, 2048) dtype=float32 (created by layer 'top_activation')>

In [10]:
from tensorflow.keras.callbacks import Callback

class Matrit(Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get("accuracy") >= 0.90 and logs.get("val_accuracy") >= 0.90:
      self.model.stop_training = True

callbacks = Matrit()

In [11]:
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomTranslation

intput = Sequential([
  RandomFlip("horizontal_and_vertical", input_shape=input_shape),
  RandomRotation(0.2),
  RandomZoom(0.2),
  RandomContrast(0.2),
  RandomTranslation(height_factor=0.2, width_factor=0.2)
])

In [12]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

model = tf.keras.models.Sequential([
  intput,
  intnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

In [13]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
plot = model.fit(train, validation_data=testi, epochs=10, callbacks=[callbacks])

In [15]:
model.evaluate(train)

[0.1774769127368927, 0.9558233022689819]

In [16]:
model.evaluate(testi)

[0.4115397334098816, 0.9312977194786072]

In [ ]:
plot = Helplot(plot, "accuracy")

plot.trainplot

In [ ]:
plot.testiplot

In [19]:
import numpy

t = "/content/image/perempuan/perempuan (280).jpg"

def output(t, input_shape=(160, 160, 3)):
  i = utils.load_img(t, target_size=input_shape)
  i = utils.img_to_array(i)
  i = tf.expand_dims(i, axis=0)
  j = model.predict(i, verbose=0)
  result = "Male" if numpy.round(j) == 1 else "Female"
  return result

output(t)

'Female'

In [20]:
t = "/content/image/pria/pria (105).jpg"

output(t)

'Male'

In [21]:
t = "/content/image/pria/pria (114).jpg"

output(t)

'Male'

In [22]:
t = "/content/image/pria/pria (126).jpg"

output(t)

'Male'

In [23]:
t = "/content/image/perempuan/perempuan (132).jpg"

output(t)

'Female'

In [24]:
t = "/content/image/perempuan/perempuan (138).jpg"

output(t)

'Female'